In [1]:
# Đọc dữ liệu từ tập tin CSV
import pandas as pd
# Đường dẫn đến tập tin CSV
file_path = 'C:\\Users\\ACER\\Desktop\\BD_DQ\\data_dotquy.csv'#Hien
#file_path = 'C:\\Users\\Admin\\Desktop\\BD_DQ\\data_dotquy.csv'
# Đọc dữ liệu từ CSV vào DataFrame
df = pd.read_csv(file_path)
# Hiển thị DataFrame trong VS Code
print(df)

         id  gender   age  hypertension  heart_disease ever_married  \
0      9046    Male  67.0             0              1          Yes   
1     51676  Female  61.0             0              0          Yes   
2     31112    Male  80.0             0              1          Yes   
3     60182  Female  49.0             0              0          Yes   
4      1665  Female  79.0             1              0          Yes   
...     ...     ...   ...           ...            ...          ...   
5105  18234  Female  80.0             1              0          Yes   
5106  44873  Female  81.0             0              0          Yes   
5107  19723  Female  35.0             0              0          Yes   
5108  37544    Male  51.0             0              0          Yes   
5109  44679  Female  44.0             0              0          Yes   

          work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0           Private          Urban             228.69  36.6  former

LÀM SẠCH và CHUẨN HÓA DL
- 1 giới tính: loại other
- 2 tuổi: Loại số thực ( vd: 0.1 , 1.56) rồi dùng kỹ thuật Age Grouping
- 3 Đã kết hôn: chuyển về 0 và một
- 4 kiểu làm việc(work_type)
    + kỹ thuật Hashing Encoding
    + kỹ thuật One-Hot Encoding
- 5 vùng sống: chuyển về 0 và một
- 6 BMI  
    + Loại bỏ các hàng hoặc cột có N/A rồi dùng kỹ thuật Binary Encoding
    + dùng kỹ thuật Simple Imputation rồi dùng kỹ thuật tiếp Min-Max Scaling
- 7 smoking_status
  + Loại bỏ các hàng hoặc cột có Unknown rồi dùng kỹ thuật tiếp Min-Max Scaling
  + kỹ thuật Frequency Encoding
- 8 avg_glucose_level (mg/dL)
    DÙNG KỸ THUẬT: Min-Max Scaling
  ******ĐIỀU KIỆN LÀ******
    Người bình thường:
     < 140
    Người Bị tiểu đường:
      >=140

In [2]:
#Hien
#Kiem tra trung lap
duplicated_rows = df[df.duplicated(subset='id')]
print("So luong du lieu trung lap la:" , len(duplicated_rows))

df = df.dropna()
df = df.applymap(lambda x: x.replace("\n"," ") if isinstance(x, str) else x )

So luong du lieu trung lap la: 0


C:\Users\ACER\AppData\Local\Temp\ipykernel_6380\2419221204.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace("\n"," ") if isinstance(x, str) else x )


In [3]:
#hien
#Kiểm tra dữ liệu có bị thiếu
df.isnull().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [4]:
#Hien
# Dung knn dien vao BMI
# Tạo đối tượng KNN Imputer với số lượng hàng xóm là 3
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)

# Áp dụng imputer cho DataFrame, chỉ chọn các cột số
df[['age', 'bmi']] = imputer.fit_transform(df[['age', 'bmi']])

# Hiển thị DataFrame sau khi điền dữ liệu
print(df)

         id  gender   age  hypertension  heart_disease ever_married  \
0      9046    Male  67.0             0              1          Yes   
2     31112    Male  80.0             0              1          Yes   
3     60182  Female  49.0             0              0          Yes   
4      1665  Female  79.0             1              0          Yes   
5     56669    Male  81.0             0              0          Yes   
...     ...     ...   ...           ...            ...          ...   
5104  14180  Female  13.0             0              0           No   
5106  44873  Female  81.0             0              0          Yes   
5107  19723  Female  35.0             0              0          Yes   
5108  37544    Male  51.0             0              0          Yes   
5109  44679  Female  44.0             0              0          Yes   

          work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0           Private          Urban             228.69  36.6  former

In [5]:
# Kiểm tra lại đã dủ dữ liệu chưa
df['bmi'].isnull().sum()

np.int64(0)

In [6]:
#Hien
# Xóa cột mã định danh
df  = df.drop(columns=['id'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4909 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4909 non-null   object 
 1   age                4909 non-null   float64
 2   hypertension       4909 non-null   int64  
 3   heart_disease      4909 non-null   int64  
 4   ever_married       4909 non-null   object 
 5   work_type          4909 non-null   object 
 6   Residence_type     4909 non-null   object 
 7   avg_glucose_level  4909 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     4909 non-null   object 
 10  stroke             4909 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 460.2+ KB


In [7]:
#Chau
#Loại bỏ giới tính mang giá trị Other
mask = df['gender'].isin(['Male', 'Female'])
df = df[mask]
print(df)

      gender   age  hypertension  heart_disease ever_married      work_type  \
0       Male  67.0             0              1          Yes        Private   
2       Male  80.0             0              1          Yes        Private   
3     Female  49.0             0              0          Yes        Private   
4     Female  79.0             1              0          Yes  Self-employed   
5       Male  81.0             0              0          Yes        Private   
...      ...   ...           ...            ...          ...            ...   
5104  Female  13.0             0              0           No       children   
5106  Female  81.0             0              0          Yes  Self-employed   
5107  Female  35.0             0              0          Yes  Self-employed   
5108    Male  51.0             0              0          Yes        Private   
5109  Female  44.0             0              0          Yes       Govt_job   

     Residence_type  avg_glucose_level   bmi   smok

In [8]:
#Chau
# gender: female 1 male 0
df.loc[:, 'gender'] = df['gender'].replace({'Female': 1, 'Male': 0})
print(df)

     gender   age  hypertension  heart_disease ever_married      work_type  \
0         0  67.0             0              1          Yes        Private   
2         0  80.0             0              1          Yes        Private   
3         1  49.0             0              0          Yes        Private   
4         1  79.0             1              0          Yes  Self-employed   
5         0  81.0             0              0          Yes        Private   
...     ...   ...           ...            ...          ...            ...   
5104      1  13.0             0              0           No       children   
5106      1  81.0             0              0          Yes  Self-employed   
5107      1  35.0             0              0          Yes  Self-employed   
5108      0  51.0             0              0          Yes        Private   
5109      1  44.0             0              0          Yes       Govt_job   

     Residence_type  avg_glucose_level   bmi   smoking_status  

C:\Users\ACER\AppData\Local\Temp\ipykernel_6380\880359247.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'gender'] = df['gender'].replace({'Female': 1, 'Male': 0})


In [ ]:
#Chau
# Residence_type: urban 1 rural 0(can note)
df.loc[:, 'Residence_type'] = df['Residence_type'].replace({'Urban': 1, 'Rural': 0})


In [ ]:
#Chau
#chuẩn hóa dữ liệu field work_type theo kỹ thuật label encoding
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df.loc[:, 'work_type'] = le.fit_transform(df['work_type'])
print(df)

In [9]:
#Chi
# Kiểm tra xem tuổi có phải là số nguyên hay không
df['Is_Whole_Number'] = df['age'].apply(lambda x: x.is_integer())

# Lọc các hàng mà tuổi không phải là số thực (chỉ giữ lại số nguyên)
df = df[df['Is_Whole_Number']]

# Xóa cột tạm thời 'Is_Whole_Number'
df = df.drop(columns=['Is_Whole_Number'])

df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
2,0,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,1,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
5,0,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5104,1,13.0,0,0,No,children,Rural,103.08,18.6,Unknown,0
5106,1,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,1,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,0,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [ ]:
#Chi
# Định nghĩa các khoảng tuổi và các nhãn cho các nhóm tuổi
bins = [0, 18, 35, 60, 100]
labels = ['0', '1', '2', '3']

# Chia cột "Age" thành các nhóm tuổi
df['Age Group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

df

In [ ]:
#Chi
# Chuyển đổi các nhóm tuổi thành các giá trị số
df['Age Group'] = df['Age Group'].cat.codes.astype(float)
df

In [ ]:
#Hien
# khong tieu duong
def glucose_normal(value):
    if value < 140:
        return value
    else:
        return 0
normal_data = [glucose_normal(x) for x in df['avg_glucose_level']]
#print(normal_data)

# Calculate min and max values
X_min = min(normal_data)
X_max = max(normal_data)

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in normal_data]
scaled_data
df['glucose_normal'] = scaled_data


In [ ]:
#Hien
# tieu duong
def glucose_classify(value):
    if value >= 140:
        return value
    else:
        return 0

classified_data = [glucose_classify(x) for x in df['avg_glucose_level']]
#classified_data

# Calculate min and max values
X_min = min(classified_data)
X_max = max(classified_data)

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in classified_data]
scaled_data
df['glucose_classify'] = scaled_data


In [ ]:
#Hien
# chuẩn hóa bằng Min-Max Scaling
glucose_values = df['avg_glucose_level']

# Calculate min and max values
X_min = min(df['avg_glucose_level'])
X_max = max(df['avg_glucose_level'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['avg_glucose_level']]

# Print the original and scaled data
#print(scaled_data)
#df['glucose_normalized_data'] = scaled_data

df['avg_glucose_level'] = scaled_data
df


In [ ]:
#Giang 

df['Residence_type']=df['Residence_type'].replace({'Urban':1,'Rural':0})
mask = df['gender'] != 'Other'
df_filtered = df[mask]
print(df_filtered) 

In [ ]:
#Giang
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['work_type'] = label_encoder.fit_transform(df['work_type'])

# sử dụng chuẩn hóa bằng Min-Max Scaling chuan hoa df['work_type'] dang (0 1 2 3 4)
# Calculate min and max values
X_min = min(df['work_type'])
X_max = max(df['work_type'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['work_type']]
scaled_data
# Tạo ra trường dữ liệu mới work_type_2
df['work_type_2'] =scaled_data
df

In [ ]:
#Hoang
# Dữ liệu ban đầu
data = df['bmi']

# Min-Max Scaling
data_min = data.min()
data_max = data.max()

scaled_data = (data - data_min) / (data_max - data_min)

# Hiển thị dữ liệu đã chuẩn hóa
print(scaled_data)

In [ ]:
#Hoang
df['bmi']=scaled_data
df

In [ ]:
#Hoang
# Dữ liệu ban đầu
data = df['bmi']

# Chuyển đổi dữ liệu thành DataFrame
dff = pd.DataFrame(data, columns=['bmi'])
dff

In [ ]:
#Hoang
import numpy as np
conditions = [
    (dff['bmi'] < 18.5),
    (dff['bmi'] >= 18.5) & (dff['bmi'] <= 25),
    (dff['bmi'] > 25)
]

# Các giá trị tương ứng cho mỗi điều kiện
df['underweight'] = np.where(dff['bmi'] < 18.5, 1, 0)
df['normal'] = np.where((dff['bmi'] >= 18.5) & (dff['bmi'] <= 25), 1, 0)
df['overweight'] = np.where(dff['bmi'] > 25, 1, 0)

# Hiển thị DataFrame với dữ liệu đã chuẩn hóa
df

In [ ]:
#Binh
df.dropna(how='all', inplace=True)
df.head()

In [ ]:
#Binh
df = df[df['smoking_status'] != 'Unknown']
df

In [ ]:
#Binh
#sử dụng labelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['smoking_status'] = le.fit_transform(df['smoking_status'])
#fit_tranform tự động gán giá trị cho smoking_status
df